##### Exercise 6


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Range di temperature
T_values = np.arange(0.5, 2.1, 0.1)

def update_input_file(temp):
    # Apri il file input.dat e riscrivi con la nuova temperatura
    with open("input.dat", "r") as file:
        lines = file.readlines()
    
    # Cerca la riga che contiene "TEMP" e sostituisci il valore
    for i, line in enumerate(lines):
        if line.startswith("TEMP"):
            lines[i] = f"TEMP                   {temp}\n"

    # Sovrascrivi il file con i nuovi dati
    with open("input.dat", "w") as file:
        file.writelines(lines)

    print(f"File input.dat aggiornato con TEMP = {temp}")



# Funzione per eseguire la simulazione
def run_simulation():
    os.system("./simulator.exe")  # Sostituisci con il comando per avviare il tuo codice.

# Liste per memorizzare i risultati
U_metro, err_U_metro = [], []
Cv_metro, err_Cv_metro = [], []
chi_metro, err_chi_metro = [], []
M_metro, err_M_metro = [], []

U_gibbs, err_U_gibbs = [], []
Cv_gibbs, err_Cv_gibbs = [], []
chi_gibbs, err_chi_gibbs = [], []
M_gibbs, err_M_gibbs = [], []

# Ciclo sulle temperature
for T in T_values:
    # Aggiorna il file di input con la temperatura corrente
    update_input_file(T)

    # Esegui la simulazione
    run_simulation()

    # Lettura dei dati del centesimo blocco per Metropolis
    data_metro = np.loadtxt("../OUTPUT/total_energy_metro.dat")
    U_metro.append(data_metro[99, 2])
    err_U_metro.append(data_metro[99, 3])

    data_Cv_metro = np.loadtxt("../OUTPUT/specific_heat_metro.dat")
    Cv_metro.append(data_Cv_metro[99, 2])
    err_Cv_metro.append(data_Cv_metro[99, 3])

    data_chi_metro = np.loadtxt("../OUTPUT/susceptibility_metro.dat")
    chi_metro.append(data_chi_metro[99, 2])
    err_chi_metro.append(data_chi_metro[99, 3])

    data_M_metro = np.loadtxt("../OUTPUT/magnetization_metro.dat")
    M_metro.append(data_M_metro[99, 2])
    err_M_metro.append(data_M_metro[99, 3])

    # Lettura dei dati del centesimo blocco per Gibbs
    data_gibbs = np.loadtxt("../OUTPUT/total_energy_gibbs.dat")
    U_gibbs.append(data_gibbs[99, 2])
    err_U_gibbs.append(data_gibbs[99, 3])

    data_Cv_gibbs = np.loadtxt("../OUTPUT/specific_heat_gibbs.dat")
    Cv_gibbs.append(data_Cv_gibbs[99, 2])
    err_Cv_gibbs.append(data_Cv_gibbs[99, 3])

    data_chi_gibbs = np.loadtxt("../OUTPUT/susceptibility_gibbs.dat")
    chi_gibbs.append(data_chi_gibbs[99, 2])
    err_chi_gibbs.append(data_chi_gibbs[99, 3])

    data_M_gibbs = np.loadtxt("../OUTPUT/magnetization_gibbs.dat")
    M_gibbs.append(data_M_gibbs[99, 2])
    err_M_gibbs.append(data_M_gibbs[99, 3])

    # Stampa dei valori letti
    print(f"T = {T}")
    print(f"Metropolis - U: {data_metro[99, 2]}, Error: {data_metro[99, 3]}")
    print(f"Gibbs - U: {data_gibbs[99, 2]}, Error: {data_gibbs[99, 3]}")

    

# Conversione in array
U_metro, err_U_metro = np.array(U_metro), np.array(err_U_metro)
Cv_metro, err_Cv_metro = np.array(Cv_metro), np.array(err_Cv_metro)
chi_metro, err_chi_metro = np.array(chi_metro), np.array(err_chi_metro)
M_metro, err_M_metro = np.array(M_metro), np.array(err_M_metro)

U_gibbs, err_U_gibbs = np.array(U_gibbs), np.array(err_U_gibbs)
Cv_gibbs, err_Cv_gibbs = np.array(Cv_gibbs), np.array(err_Cv_gibbs)
chi_gibbs, err_chi_gibbs = np.array(chi_gibbs), np.array(err_chi_gibbs)
M_gibbs, err_M_gibbs = np.array(M_gibbs), np.array(err_M_gibbs)

# --- Fitting Modelli Teorici ---
# Qui puoi aggiungere i tuoi modelli teorici (esempio: curve teoriche per U, Cv, chi, M)
def theoretical_model(T):
    # Esempio: sostituisci con il tuo modello teorico
    return T**2, T, np.sqrt(T), T**3

U_theory, Cv_theory, chi_theory, M_theory = theoretical_model(T_values)

# --- Grafici ---
fig, axs = plt.subplots(4, 2, figsize=(20, 25))
fig.suptitle("1D Ising Model - Metropolis vs Gibbs", fontsize=24)

# Grafici per ciascuna variabile
variables = [
    (U_metro, err_U_metro, U_gibbs, err_U_gibbs, U_theory, "Internal Energy"),
    (Cv_metro, err_Cv_metro, Cv_gibbs, err_Cv_gibbs, Cv_theory, "Heat Capacity"),
    (chi_metro, err_chi_metro, chi_gibbs, err_chi_gibbs, chi_theory, "Susceptibility"),
    (M_metro, err_M_metro, M_gibbs, err_M_gibbs, M_theory, "Magnetization")
]

for i, (metro, err_metro, gibbs, err_gibbs, theory, title) in enumerate(variables):
    # Metropolis
    axs[i, 0].errorbar(T_values, metro, yerr=err_metro, capsize=3, label="Metropolis")
    axs[i, 0].plot(T_values, theory, '--', label="Theoretical model")
    axs[i, 0].set_title(f"{title} - Metropolis")
    axs[i, 0].set_xlabel("Temperature")
    axs[i, 0].set_ylabel(title)
    axs[i, 0].legend()

    # Gibbs
    axs[i, 1].errorbar(T_values, gibbs, yerr=err_gibbs, capsize=3, label="Gibbs")
    axs[i, 1].plot(T_values, theory, '--', label="Theoretical model")
    axs[i, 1].set_title(f"{title} - Gibbs")
    axs[i, 1].set_xlabel("Temperature")
    axs[i, 1].set_ylabel(title)
    axs[i, 1].legend()

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
T, E, err_E = np.loadtxt("metropolis_00.dat", usecols=(0,1,2), unpack='true')
T, CV, err_CV = np.loadtxt("metropolis_00.dat", usecols=(0,3,4), unpack='true')
T, S, err_S = np.loadtxt("metropolis_00.dat", usecols=(0,5,6), unpack='true')
T, M, err_M = np.loadtxt("metropolis_02.dat", usecols=(0,1,2), unpack='true')

figure, axis = plt.subplots(2, 2)
figure.set_figwidth (20)
figure.set_figheight (20)

axis [0, 0].errorbar(T, E, yerr = err_E)
axis [0, 0].set_title("tot energy")

axis [0, 1].errorbar(T, CV, yerr = err_CV)
axis [0, 1].set_title("specitic heat")

axis [1, 0].errorbar(T, S, yerr = err_S)
axis [1, 0].set_title("susceptibility")

axis [1, 1].errorbar(T, M, yerr = err_M)
axis [1, 1].set_title("magnetization (h = 0.02)")

plt.show()

In [ ]:
# Load data
T, E, err_E = np.loadtxt("metropolis_00.dat", usecols=(0,1,2), unpack=True)
T, CV, err_CV = np.loadtxt("metropolis_00.dat", usecols=(0,3,4), unpack=True)
T, S, err_S = np.loadtxt("metropolis_00.dat", usecols=(0,5,6), unpack=True)
T, M, err_M = np.loadtxt("metropolis_02.dat", usecols=(0,1,2), unpack=True)

fig, axs = plt.subplots(2, 2, figsize=(20, 20))
fig.suptitle("1D Ising Model Simulation Results", fontsize=24)

# Internal Energy
axs[0, 0].errorbar(T, E, yerr=err_E, capsize=3, ecolor='red', markeredgecolor='black', label='Simulation')
axs[0, 0].plot(x, yU, '--', label='Theoretical model', color='orange')
axs[0, 0].set_title("Internal Energy", fontsize=18)
axs[0, 0].set_xlabel("Temperature (K)", fontsize=14)
axs[0, 0].set_ylabel("Energy", fontsize=14)
axs[0, 0].legend()

# Specific Heat
axs[0, 1].errorbar(T, CV, yerr=err_CV, capsize=3, ecolor='red', markeredgecolor='black', label='Simulation')
axs[0, 1].plot(x, yH, '--', label='Theoretical model', color='orange')
axs[0, 1].set_title("Heat Capacity", fontsize=18)
axs[0, 1].set_xlabel("Temperature (K)", fontsize=14)
axs[0, 1].set_ylabel("C", fontsize=14)
axs[0, 1].legend()

# Susceptibility
axs[1, 0].errorbar(T, S, yerr=err_S, capsize=3, ecolor='red', markeredgecolor='black', label='Simulation')
axs[1, 0].plot(x, yX, '--', label='Theoretical model', color='orange')
axs[1, 0].set_title("Susceptibility", fontsize=18)
axs[1, 0].set_xlabel("Temperature (K)", fontsize=14)
axs[1, 0].set_ylabel("χ", fontsize=14)
axs[1, 0].legend()

# Magnetization
axs[1, 1].errorbar(T, M, yerr=err_M, capsize=3, ecolor='red', markeredgecolor='black', label='Simulation')
axs[1, 1].plot(x, yM, '--', label='Theoretical model', color='orange')
axs[1, 1].set_title("Magnetization (h = 0.02)", fontsize=18)
axs[1, 1].set_xlabel("Temperature (K)", fontsize=14)
axs[1, 1].set_ylabel("Magnetization", fontsize=14)
axs[1, 1].legend()

for ax in axs.flat:
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.tick_params(labelsize=12)

plt.tight_layout()
plt.show()